## Exploring raw sleep study data

In [ ]:
import os
import pyedflib
import numpy as np
import json

import torch
torch.manual_seed(42)
import torch.nn as nn
from torch.utils.data import TensorDataset, Dataset
from torch.autograd import Variable

### Load raw EEG/Annotation data

In [6]:
with open('../../data/interim/target.json') as f:
    targets = json.load(f)

sleep_cassette_files = os.listdir('../../data/raw/sleep-cassette/')

eeg_list = []
annotation_list = []

for target in targets:
    # Find individual patient
    subject_id_str = str(target['subject'])
    if len(subject_id_str) == 1:
        subject_id_str = '0' + subject_id_str
    night_str = str(target['night'])
    
    # Whole night sleep recordings containing EEG
    eeg = pyedflib.EdfReader('../../data/raw/sleep-cassette/SC4' + subject_id_str + night_str + 'E0-PSG.edf')
    eeg_list.append(np.array(eeg.readSignal(0)))
    
    # Annotations of sleep recordings corresponding to EEG
    annotation_file_path = list(filter(lambda x: x.startswith('SC4' + subject_id_str + night_str) and x.endswith('-Hypnogram.edf'), sleep_cassette_files))[0]
    annotation = pyedflib.EdfReader('../../data/raw/sleep-cassette/' + annotation_file_path)
    annotation_list.append(annotation)

### Check dimensionality
Mirror pre-processing done by Tsinalis by ignoring 'Sleep stage W' and 'Sleep stage ?'

In [3]:
# Check extreme bounds
eeg_min = np.hstack(eeg_list).min()
eeg_max = np.hstack(eeg_list).max()
display(f'Min: {eeg_min}')
display(f'Max: {eeg_max}')

'Min: -208.0'

'Max: 209.0'

In [4]:
# Count total number of epochs across patients
filtered_stages = np.array([
    'Sleep stage 1', 'Sleep stage 2', 'Sleep stage 3',
    'Sleep stage 4', 'Sleep stage R'])

total_epochs = 0
for annotation in annotation_list:
    training_epochs = 0
    time_marks = annotation.readAnnotations()[0]
    stages = annotation.readAnnotations()[2]
    qualifying_indices = np.where(np.isin(stages, filtered_stages))[0]
    
    for index in qualifying_indices:
        training_epochs += (time_marks[index + 1] - time_marks[index]) / 30
    total_epochs += training_epochs

print(f'Total epoch count: {total_epochs}')

Total epoch count: 34023.0


### Building training set of 2D matrices
For each of the 39 patients, their EEG values are walked through in batches of 15,000 (5 epochs: the current epoch both preceeded and succeeded by 2 epochs). These 1D arrays of length 15,000 are then converted in an 417 by 15,000 2D matrix. 417 is the absolute difference between the min (-208) and the max (209) EEG values. 

For each of the 15,000 non-negative integers in the 1-D array, we fill in the i-th row and j-th column of a 417 by 15,000 2D array with the value of 1, where i is the value of the non-negative integer and j is the index of the non-negative integer in the 15,000 non-negative integer array. The resulting 417 by 15,000 2D array (2D image) will then be the final input used to train a CNN with the corresponding sleeping stage label.

In [7]:
def get_sleep_stage(index, time_marks, sleep_stages):
    """Determines the sleep stage label for a patient
    at a given point in their nightly data.
    
    index : Integer
        Current indidvial EEG reading location from 1-D array
    
    time_marks : Array-like
        Breakpoints denoting when different sleep stages
        occur. Array values are assumed to be starting
        times for each corresponding sleep stage
    
    sleep_stages : Array-like
        Corresponding sleep stage labels that match the
        breaks denoted by `time_marks`
    
    Returns : String
        Sleep stage label
    """
    time_index = -1
    for time_mark in time_marks:
        if index >= time_mark:
            time_index += 1
        else:
            break
    return sleep_stages[time_index]

Build the individual matrices to be later passed into the CNN. When walking through nightly EEG data, skip over the first two epochs (i.e. 6000 items) so that the first epoch that matrix is built from is able to incorporate it's previous two epochs. Likewise, stop walking through EEG data two epochs early.

In [8]:
images = []
for subject in range(len(eeg_list)):
    
    # Collect subject's raw data and labels
    eeg = eeg_list[subject]
    annotation = annotation_list[0]
    
    # Gather timing breakpoints for each sleep stage
    sleep_stages = annotation.readAnnotations()[2]
    time_marks = annotation.readAnnotations()[0]
    time_marks *= 100  # Convert from 100Hz to Hz, matching the EEG indices
    
    # Iterate through nightly data to create 2D matrices
    start_idx = 6000
    stop_idx = 9000
    two_epochs = 6000
    for index in range(6000, eeg.shape[0] - 6000, 3000):

        # Record target sleep stage label
        label = get_sleep_stage(index, time_marks, sleep_stages)
        if label == 'Sleep stage 4':
            label = 'Sleep stage 3'
        if label == 'Sleep stage ?':
            continue

        # Shift EEG values to ensure all are positive integers
        eeg_vals = eeg[(start_idx - two_epochs):(stop_idx + two_epochs)]
        eeg_vals_adjust = np.apply_along_axis(lambda x: np.round(x + 208), axis=0, arr=eeg_vals)

        # Mark locations in the 2d array of EEG values
        image = np.zeros((417, 15000), dtype='int')
        for j, eeg_val in enumerate(eeg_vals_adjust):
            image[int(eeg_val), j] = 1

        # Save patient matrix with training label
        images.append([image, label])

        # Reset slicing indices
        start_idx += 3000
        stop_idx += 3000
    
    break  # Remove for full patient run

### Format data for CNN input

In [9]:
# View dimensions for original data
label_map = {
    'Sleep stage 1': 1,
    'Sleep stage 2': 2,
    'Sleep stage 3': 3,
    'Sleep stage R': 4,
    'Sleep stage W': 5
}

X_fullset = []
y_fullset = []
i = 0
for image in images:
    X_fullset.append(image[0])
    y_fullset.append(np.array([label_map[image[1]]]))
    i += 1
    if i > 3:
        break

X_fullset = np.array(X_fullset)
y_fullset = np.array(y_fullset)

display(f'X shape: {X_fullset.shape}')
display(f'y shape: {y_fullset.shape}')

'X shape: (4, 417, 15000)'

'y shape: (4, 1)'

In [24]:
# View dimensions as data are passed into PyTorch
data = torch.from_numpy(X_fullset.astype('float32')).unsqueeze(1)
target = torch.from_numpy(y_fullset.astype('long'))
dataset = TensorDataset(data, target)
display(f'X shape: {data.shape}')
display(f'y shape: {target.shape}')

'X shape: torch.Size([4, 1, 417, 15000])'

'y shape: torch.Size([4, 1])'

In [25]:
# Display sample to ensure correct loading into PyTorch
dl = torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=True)

dataiter = iter(dl)
X_samples, y_samples = dataiter.next()
print(X_samples)
print(y_samples)

tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]]])
tensor([[5]])


### Build and run CNN

In [22]:
class SleepCNN(nn.Module):
    def __init__(self):
        super(SleepCNN, self).__init__()
        self.conv_1 = nn.Conv2d(in_channels=1, out_channels=20, kernel_size=(200, 200))
        self.pool_1 = nn.MaxPool2d(kernel_size=20)
        self.conv_2 = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=1)
        self.pool_2 = nn.MaxPool2d(kernel_size=1)
        self.fc = nn.Linear(in_features=20, out_features=5)
    
    def forward(self, x):
        print(f'Enter: {x.shape}')
        x = self.conv_1(x)
        print(f'Finished Convolution: {x.shape}')
        x = nn.functional.relu(x)
        print(f'Finished RELU: {x.shape}')
        x = self.pool_1(x)
        print(f'Finished Pool: {x.shape}')
#         x = nn.functional.relu(self.conv_2(x))
#         x = self.pool_2(x)
        x = self.fc(x)
        return x


model = SleepCNN()

In [ ]:
output = model(Variable(X_samples))
output

Enter: torch.Size([1, 1, 417, 15000])
